## Can we improve our performence by using several retrievers ?

In [1]:
import pandas as pd 
from harvai.bm25 import Bm25
from harvai.embedding import Embedding
from harvai.nn_model import Nn_model
from harvai.dpr import DPR
from harvai.utils import score
import seaborn as sns
import matplotlib.pyplot as plt

INFO - haystack.document_stores.base -  Numba not found, replacing njit() with no-op implementation. Enable it with 'pip install numba'.
INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
ERROR - root -  Failed to import 'magic' (from 'python-magic' and 'python-magic-bin' on Windows). FileTypeClassifier will not perform mimetype detection on extensionless files. Please make sure the necessary OS libraries are installed if you need this functionality.


In [2]:
article_number = 5

In [19]:
bm25 = Bm25(10)
bm25.clean_data()
bm25.fit()

In [20]:
dpr = DPR(10)
dpr.clean_data()
dpr.fit()

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'CamembertTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Model found locally at ../raw_data/retriever.pt/query_encoder
WARNING - haystack.modeling.model.language_model -  Using a model of type 'camembert' which might be incompatible with DPR encoders.Bert based encoders are supported that need input_ids,token_type_ids,attention_mask as input tensors.
INFO - haystack.modeling.model.language_model -  Loaded ../raw_data/retriever.pt/query_encoder
The tokenizer class

In [21]:
nn_model = Nn_model(10)
nn_model.clean_data()
nn_model.fit()

In [22]:
embedding = Embedding(10)
embedding.clean_data()
embedding.fit()

In [7]:
question = "quelle est la vitesse maximum sur l'autoroute ?"

In [8]:
embedding.predict(question)

In [9]:
bm25.articles

In [10]:
bm25.predict(question)

In [11]:
bm25.articles

[669, 688, 976, 6, 403]

In [12]:
output = []
for article in bm25.articles: 
    if article in embedding.articles:
        output.append(article)

if len(output) == 0: 
    output = bm25.articles

In [13]:
output

[669, 688, 976, 6, 403]

In [28]:
class MixRetriever: 
    def __init__(self,retriever1,retriever2):
        self.retriever1 = retriever1
        self.retriever2 = retriever2
        self.articles = None
    
    def predict(self,question):
        self.retriever1.predict(question)
        self.retriever2.predict(question)
     
        output =[]
        for article in self.retriever1.articles: 
            if article in self.retriever2.articles:
                output.append(article)

        if len(output) == 0: 
            output = self.retriever1.articles

        self.articles = output
        

In [29]:
retriever_mutant = MixRetriever(dpr,bm25)

In [30]:
score(retriever_mutant)

100%|██████████| 51/51 [00:09<00:00,  5.46it/s]


{'recall': 0.5294117647058824, 'average rank': 1.4074074074074074}

In [25]:
score(bm25)

100%|██████████| 51/51 [00:00<00:00, 226.59it/s]


{'recall': 0.8235294117647058, 'average rank': 2.1666666666666665}